In [1]:
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.query import MetadataQuery

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

# Initialize clients
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
print(f"weaviate client connected: {client.is_connected()}")

weaviate client connected: True


In [2]:
query = "How do I get in touch with support?"

response = openai_client.embeddings.create(model="text-embedding-3-large",input=query)
query_embedding = response.data[0].embedding

In [7]:
COLLECTION_NAME = "TeslaCybertruckOwnersManual"

collection = client.collections.get(COLLECTION_NAME)
similar_texts = collection.query.near_vector(
    near_vector=query_embedding,
    limit=3,
    return_properties=["text"],
    return_metadata=MetadataQuery(distance=True)
)

print(f"\nRetrieved {len(similar_texts.objects)} contexts for query '{query}':")
for i, doc in enumerate(similar_texts.objects, 1):
    print(f"\n{i}. Distance: {doc.metadata.distance}")
    print(f"Text: {doc.properties['text']}\n\n")



Retrieved 3 contexts for query 'How do I get in touch with support?':

1. Distance: 0.6159154176712036
Text: Technical Support

If you need further assistance, contact the Tesla Support team via the Contact Us page:

tesla.com/support/energy/more/additional-support/contact-us

Have the following information available when contacting Tesla:

- Owner name
- Best way for Tesla to contact you (name, phone number, email)
- Powershare Gateway serial number
- Brief description of the issue

Charging and Energy Consumption                                187
---



2. Distance: 0.6914113759994507
Text: Contacting Tesla

For detailed information about your Cybertruck, go to http://www.tesla.com and log on to your Tesla account or sign up to get an account.

If you have any questions or concerns about your Cybertruck, in the United States, Canada or Puerto Rico, call 1-877-79TESLA (1-877-798-3752) and in Mexico, call 1-800-228-8145.

NOTE: You can also use voice commands to provide feedback to T

In [8]:

context_str = "\n\n---\n\n".join(doc.properties["text"] for doc in similar_texts.objects)

prompt = f"""Answer the question using ONLY the information provided in the context below. 
Do not add any general knowledge or information not contained in the context."

Context:
{context_str}

Question: {query}

Answer:"""

# 6. Generate answer using GPT-4
response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

answer = response.choices[0].message.content

# 7. Print results
print(f"Prompt: {prompt} \n\n----------------------------------\n\n")
print(f"Question: {query}\n\nAnswer: {answer}")

Prompt: Answer the question using ONLY the information provided in the context below. 
Do not add any general knowledge or information not contained in the context."

Context:
Technical Support

If you need further assistance, contact the Tesla Support team via the Contact Us page:

tesla.com/support/energy/more/additional-support/contact-us

Have the following information available when contacting Tesla:

- Owner name
- Best way for Tesla to contact you (name, phone number, email)
- Powershare Gateway serial number
- Brief description of the issue

Charging and Energy Consumption                                187
---

---

Contacting Tesla

For detailed information about your Cybertruck, go to http://www.tesla.com and log on to your Tesla account or sign up to get an account.

If you have any questions or concerns about your Cybertruck, in the United States, Canada or Puerto Rico, call 1-877-79TESLA (1-877-798-3752) and in Mexico, call 1-800-228-8145.

NOTE: You can also use voice co